## DSCI 524 - Collaborative Software Development

### Lecture 4: Debugging and package documentation

#### 2020-03-04

## Lecture 4 learning objectives:
By the end of this lecture, students should be able to:

- Use `traceback`, `options(error = recover)`, and `browser` to debug R code
- Use `traceback.print_tb`, `pdb.pm()`, `code.interact()` and `pdb.run` to debug Python code
- Generate well formatted function and package-level documentation for R and Python packages using Roxygen2, `pkgdown` and Sphinx & Read the Docs